In [ ]:
!pip install -q -U google-generativeai
!pip install langchain
!pip install unstructured
!pip install pdf2image
!pip install pdfminer.six
!pip install pinecone-client
!pip install pikepdf
!pip install pypdf
!pip install unstructured-inference

In [ ]:
import google.generativeai as genai
import os

gemini_api_key = ""
genai.configure(api_key = gemini_api_key)

# Uploading embeddings to pinecone

In [ ]:
from pinecone import Pinecone

pc = Pinecone(api_key="")
index = pc.Index("data")

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Pinecone
loader = UnstructuredPDFLoader("48lawsofpower.pdf")
data = loader.load()
text_splitter =  CharacterTextSplitter(separator = ".",chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)



In [ ]:
vector_list=[]
x=0
for i in texts:
  x=x+1
  li=[]
  output = genai.embed_content(
    model="models/embedding-001",
    content=i.page_content,
    task_type="retrieval_document",
    title="Embedding of document 1")
  embedding=output['embedding']
  li={
      "id":str(x),
      "values": embedding,
      "metadata": {"content":i.page_content}
  }
  vector_list.append(li)
print(vector_list)

[{'id': '1', 'values': [0.0005923026, -0.010348844, -0.022866858, -0.017666515, 0.06691313, -0.0129924705, -0.029910846, -0.0076627512, 0.016270244, 0.021356238, 0.061234575, 0.073280245, 0.020779276, -0.03326323, -0.023642093, -0.019318083, 0.018546833, 0.060968556, 0.028689949, -0.058083717, -0.039586384, 0.0090028485, 0.0068624225, -0.004298265, -0.0074661025, -0.006557903, 0.042322427, -0.08919441, 0.02278153, 0.016484112, -0.044300232, 0.042130582, -0.030716648, -0.010625776, 0.016126633, -0.05956875, -0.023144957, 0.028478991, -0.06365982, 0.029618673, 0.021707904, 0.031308774, -0.008097769, -0.028934155, 0.014102197, -0.029426914, -0.04446265, 0.0011598623, -0.013520596, -0.06863535, -0.03258065, -0.012424336, 0.037140403, -0.04348399, 0.03943953, 0.006600969, 0.06651182, -0.017799536, -0.0734526, -0.034464665, -0.023856439, -0.004522673, -0.015089013, 0.010906349, 0.024242396, -0.024537886, -0.036384452, 0.027603999, 0.06673652, -0.01055568, 0.028652962, -0.011879964, 0.0836437

In [ ]:
index.upsert(
    vectors=vector_list)

{'upserted_count': 51}

# Querying the document

In [ ]:
question="What's the point of making myself less accessible?"
encoded_query=genai.embed_content(
    model="models/embedding-001",
    content=question,
    task_type="retrieval_query")

res=index.query(vector=encoded_query["embedding"],top_k=10,include_metadata=True)
paragraphs=[]
final_paragraph=""
for passage in res['matches']:
  final_paragraph+=passage['metadata']['content']+"/n/n"
response = model.generate_content(
    f"""You are provided with some passages from a book along with a question. Answer the user's question from the supporting passages of the book.

    Question: {question}

    Supporting passages:
    {final_paragraph}
""")
print(response.text)

Making yourself less accessible creates value through scarcity. If you are always available, people will take you for granted and your presence will become routine. By making yourself less accessible, you create an aura of exclusivity and mystery that will make people want to be around you more. This can be done by limiting your availability, being selective about who you spend time with, and avoiding oversharing personal information.
